# Wikipedia Research Agent

This notebook demonstrates how to create an Azure AI Foundry agent that performs research using only Wikipedia as its information source. The agent leverages a dedicated OpenAPI tool defined by `wikipedia_search_api_openapi_spec.json` to search and retrieve information from Wikipedia articles.

## Install dependencies (if needed)

In [1]:
# !pip install -r ../../requirements.txt

## Import required packages

In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
import jsonref
from azure.ai.agents.models import OpenApiTool, OpenApiAnonymousAuthDetails
from dotenv import load_dotenv

load_dotenv('../../.env', override=True)

True

## Load and patch OpenAPI spec for Wikipedia Search Tool

In [3]:
with open("wikipedia_search_api_openapi_spec.json", "r") as f:
    openapi_spec = jsonref.loads(f.read())
# Patch the servers attribute to use the runtime API_BASE_URL
openapi_spec["servers"] = [ {"url": os.environ["API_BASE_URL"]} ]

auth = OpenApiAnonymousAuthDetails()
wikipedia_tool = OpenApiTool(
    name="wikipedia_search",
    spec=openapi_spec,
    description="Search and retrieve information from Wikipedia articles.",
    auth=auth
)

## Create Azure AI Project client

In [4]:
project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_FOUNDRY_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

## Define and create the Wikipedia Research Agent

In [5]:
agent = project_client.agents.create_agent(
    model="gpt-4.1",
    name="wikipedia-research-agent",
    instructions=f"""# Wikipedia Research Agent System Message

You are a focused and efficient research assistant. Your task is to use the Wikipedia Search Tool to research any topic or question provided by the user. For each request, follow these steps:

---

## 1. Research and Retrieval
- Use the Wikipedia Search Tool to search Wikipedia for the most relevant and up-to-date information on the user's topic or question.
- Formulate effective queries and refine them as needed to maximize the quality of your findings.

---

## 2. Key Learnings Output
- Summarize your findings as a list of **5 key learnings** (bulleted list).
- Each key learning must be detailed, factual, and directly related to the user's topic.
- **Attach a source citation** (Wikipedia article URL) to each bullet point, formatted as `[source]`.

---

## 3. Communication Style
- Be clear, neutral, and informative.
- Do not speculate or include unsupported information.
- If you cannot find sufficient information, state this politely and suggest clarifying the query if possible.

---

## 4. Constraints
- Only use information retrieved via the Wikipedia Search Tool.
- Do not include personal opinions or unverified content.
- Always provide a source for each key learning.

By following these guidelines, you will deliver concise, well-sourced research summaries for any topic or question, strictly using Wikipedia as your information source.
""",
    temperature=0.0,
    tools=wikipedia_tool.definitions,
    headers={"x-ms-enable-preview": "true"}
)
agent_id = agent.id
print(f"Agent ID: {agent_id}")

Agent ID: asst_40ugrWZ2LFCA26CfmFuoQk5v
